# EXECUTABLE: LONGITUDINAL PREDICTION IN PRIMARY CARE (PC)

The main objective of this notebook is to carry out a time-series forecasting analysis to evaluate and identify diagnostic models that could be useful for predicting others. Previous studies have shown that different types of variables — such as atmospheric, temporal, and mobility variables, among others — can help anticipate increases in population interactions with the healthcare system.

These interactions, which mainly take the form of medical visits, are associated with one or more coded diagnoses. This study poses the following working hypothesis:

    - Is it possible to accurately predict the number of diagnoses associated with medical visits using past diagnoses as predictor variables?

The ultimate goal is to determine which diagnoses have predictive power over others and, consequently, whether they can be used as early indicators of specific pathologies. This knowledge could be especially relevant for:

- optimizing healthcare resource planning,
- detecting seasonal or emerging disease patterns, and
- reinforcing preventive strategies within the scope of primary care.

This notebook therefore constitutes a first executive approach to longitudinal prediction based on diagnoses. 

In [ ]:
import shutil
shutil.rmtree('__pycache__', ignore_errors=True)

In [ ]:
import pandas as pd
import numpy as np
import math

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import scipy.stats
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import logging
logging.disable(logging.CRITICAL)

In [ ]:
import torch.optim as optim
from darts.models import RNNModel, Theta
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, Theta
from darts.metrics import mape, rmse, r2_score
from darts.utils.statistics import check_seasonality, plot_acf
from darts.utils.timeseries_generation import datetime_attribute_timeseries

from darts.datasets import SunspotsDataset, AirPassengersDataset
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
import sys
import time

## 1. Data Reading and Processing

The algorithm uses temporal diagnostic databases of shape m × n, where m refers to the number of days and n refers to the different diagnoses.

From the original dataset, two key transformations are generated:

- 1. Smoothed Dataset: Each time series in the database is transformed to become smoother, allowing a clearer view of the underlying trend. The main goal is to eliminate spikes caused by the weekend effect (when there are no visits to primary care and, therefore, no diagnoses are recorded). Additionally, data scaling is applied so that the minimum and maximum values are normalized between 0 and 1. This facilitates the interpretation of performance metrics.

- 2. Infection Index Dataset: Some diagnoses refer to infectious diseases, which typically do not behave randomly but instead follow seasonal patterns. The objective of this transformation is to compute an infection risk index, based on the diagnostic activity in the previous 10 days. This aims to capture short-term temporal dependencies associated with infectious outbreaks.

In [ ]:
# smoother function ---------------------------------
#       (revisar --> rolling(funció per fer-ho més fàcil))
def smoother(df,window_size):
    smoothed = {}
    for column in df.columns:
        if column != 'visi_data_visita':
            column_list = []
            for i in range(len(df)):
                if i == 0:
                    column_list.append(df[column].iloc[0])
                elif i < window_size:
                    column_list.append(df[column].iloc[:i+1].mean())
                else:
                    column_list.append(df[column].iloc[i-window_size:i+1].mean())
            smoothed[column] = column_list

    smoothed = pd.DataFrame.from_dict(smoothed)
    smoothed.set_index(df.index, inplace=True)
    return smoothed

# infection index function -------------------
def infection_index_df(df,prior_days):
    infection_index = {}
    cols = list(df.columns)
    for column in cols:
        column_list = []
        for i in range(len(df)):
            if i < prior_days:
                column_list.append(float('nan'))
            else:
                if sum(df[column].iloc[(i-prior_days):(i-1)]) > 0:
                    inf_idx = df[column].iloc[i] / sum(df[column].iloc[(i-prior_days):(i-1)])
                else:
                    inf_idx = float('nan')
                column_list.append(inf_idx)
        infection_index[column] = column_list
    infection_index = pd.DataFrame.from_dict(infection_index)
    infection_index.set_index(df.index, inplace=True)
    return infection_index


# plot example 10 diags -------------------------
def plot_example(df, title):
    """
    Randomly selects and plots 10 time series from a DataFrame.
    
    Parameters:
    - df (pd.DataFrame): Time-indexed DataFrame with multiple columns (e.g. diagnostics).
    - title (str): Title for the plot.
    """
    # Randomly sample 10 columns
    sampled_cols = np.random.choice(df.columns, size=10, replace=False)
    dff = df[sampled_cols].copy()

    # Add date column
    dff["date"] = dff.index

    # Melt and plot
    sns.set(rc={'figure.figsize': (20, 8)})
    sns.lineplot(
        data=dff.melt(id_vars=['date']),
        x='date', y='value', hue='variable'
    ).set(title=title)

    plt.xlabel("Date")
    plt.ylabel("Value")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
# plot example 10 diags -------------------------
def ploter(df,title,n,code):
    # PLOT RAW DATA (example 10 diags)
    cols = list(df.columns)[0:n]
    cols.append(code)
    dff = df[cols]
    dff["date"] = dff.index
    sns.set(rc={'figure.figsize':(20,8)})
    sns.lineplot(data=dff.replace('nan', float('nan')).melt(id_vars=['date']),x='date', y='value', hue='variable').set(title=title)

### 1.1 Databases plots (10 example diagnoses)

In [ ]:
# RAW DATA --------------------------------------------------------------
df = pd.read_csv('synthetic_timeseries.csv', index_col=0)
df.index = pd.date_range(start="2010-01-01", periods=len(df), freq="D")
df = df.clip(lower=0)
df

In [ ]:
plot_example(df,"RAW DATA (example 10 ts)")

In [ ]:
# SMOOTHED DATA --------------------------------------------------------
window_size = 14
smoothed = smoother(df, window_size)
plot_example(smoothed,"SMOOTHED (example 10 ts)")

In [ ]:
smoothed_scaled = (smoothed-smoothed.min())/(smoothed.max()-smoothed.min()) # normalize df
plot_example(smoothed_scaled,"SMOOTHED + SCALED DATA (example 10 ts)")

In [ ]:
smoothed = smoothed_scaled

## 2. Cross-Correlation Analysis

The objective at this stage is to identify which diagnoses behave similarly, and therefore may provide redundant information to the predictive models. The ultimate goal is to define models that are as simple and parsimonious as possible — that is, models that use the smallest number of variables while still accurately predicting the target diagnosis.

At this point, we define the target variable to be predicted based on the following parameters:

- 1. code: The code corresponding to the diagnosis we aim to predict (i.e., the dependent variable).
- 2. correlation_threshold: The maximum allowed correlation between variables. Variables with a correlation above 90% with any other are considered redundant and excluded from the model.

This step ensures that the input features used for training are informative but not collinear, thus improving both model performance and interpretability.

In [ ]:
#obtenir variables correlacionades ------------------------------------------
# WARNING = No utilitzar scaled data
def get_top_correlations_blog(df, threshold=0.90):
    orig_corr = df.corr()
    c = orig_corr.abs()
    so = c.unstack()

    i = 0
    pairs = set()
    result = pd.DataFrame()

    for index, value in so.sort_values(ascending=False).items():  # <-- FIXED HERE
        # Exclude duplicates and self-correlations
        if (
            value > threshold
            and index[0] != index[1]
            and (index[0], index[1]) not in pairs
            and (index[1], index[0]) not in pairs
        ):
            result.loc[i, ['Variable 1', 'Variable 2', 'Correlation Coefficient']] = [
                index[0],
                index[1],
                orig_corr.loc[index[0], index[1]],  # cleaner tuple access
            ]
            pairs.add((index[0], index[1]))
            i += 1

    return result.reset_index(drop=True).set_index(['Variable 1', 'Variable 2'])

# Reduir el nombre de variables correlacionades ------------------------------
def compute_vif(considered_features, df):
    
    X = df[considered_features]
    # the calculation of variance inflation requires a constant
    X['intercept'] = 1
    
    # create dataframe to store vif values
    vif = pd.DataFrame()
    vif["Variable"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif = vif[vif['Variable']!='intercept']
    return vif

# filter variables based on VIF threshold -------------------------------
def filter_VIF(vif,dataframe,iterations_max,VIF_threshold):
    iterations = 0
    while vif['VIF'].iloc[0] > VIF_threshold and iterations < iterations_max:
        redundant_vars.pop(0)
        vif = compute_vif(redundant_vars, dataframe).sort_values('VIF', ascending=False)
        
        if iterations % 5 == 0:
            print("... iteration ==> "+str(iterations)+" max VIF ==> "+str(vif.iloc[vif['VIF'].idxmax()]["VIF"]))
        
        iterations += 1
    
    print(">>> VIF variables surpassing thresholds ....")
    display(vif)
    
    return redundant_vars

In [ ]:
# DIAG
code = 'timeseries_350'

# CORR
correlation_threshold = 0.90

# VIF
iterations_max = 400
iterations = 0
VIF_threshold = 20.0

1. Computem tots els coeficients de correlació entre les variables d'estudi i ens quedem amb aquells que superen un llindar de 0.9

In [ ]:
# Make a copy of the smoothed dataframe
dataframe = smoothed.copy()

# Compute top correlations above a threshold
df_correlations = get_top_correlations_blog(dataframe, threshold=correlation_threshold)
display(df_correlations)

# Extract correlated variable names from both levels of the MultiIndex
correlated_vars = list(set(df_correlations.index.get_level_values(0).tolist() + 
                           df_correlations.index.get_level_values(1).tolist()))

# Compute variables not involved in any strong correlation
non_correlated_vars = list(set(dataframe.columns) - set(correlated_vars))

2. Computem els valors VIF de les variables i reduïm les variables amb alta correlació.

In [ ]:
# variables que són subjecte de ser eliminades
redundant_vars = correlated_vars

# computar el vif de cadascuna
vif = compute_vif(redundant_vars, dataframe).sort_values('VIF', ascending=False)
display(vif)

# filtrar el vif
redundant_vars = filter_VIF(vif,dataframe,iterations_max, VIF_threshold)

3. Exemple de reducció de la matriu de correlacions (només funciona quan el nombre de variables és inferior a n):

In [ ]:
n = 100

if dataframe.shape[1] < n:
    # ABANS -----------------------------------------------------
    # set figure size 
    plt.figure(figsize=(20,12))
    # Generate a mask to onlyshow the bottom triangle
    mask = np.triu(np.ones_like(dataframe.corr(), dtype=bool))
    # generate heatmap
    sns.heatmap(dataframe.corr(), annot=True, mask=mask, vmin=-1, vmax=1)
    plt.title('Correlation Coefficient Of Predictors')
    plt.show()


    # DESPRÉS -------------------------------------------------
    interesting_vars = redundant_vars+non_correlated_vars
    # set figure size
    plt.figure(figsize=(20,12))
    # Generate a mask to onlyshow the bottom triangle
    mask = np.triu(np.ones_like(dataframe[interesting_vars].corr(), dtype=bool))
    # generate heatmap
    sns.heatmap(dataframe[interesting_vars].corr(), annot=True, mask=mask, vmin=-1, vmax=1)
    plt.title('Correlation Coefficient Of Predictors')
    plt.show

In [ ]:
# subset of dataframe with interesting vars
print(">>> Initial number of variables in dataframe ... %s" % dataframe.shape[1])
interesting_vars = redundant_vars+non_correlated_vars 
dataframe = dataframe[interesting_vars]
dataframe[code] = smoothed[code] # tornem a addherir la variable a predir
print(">>> Final number of variables in dataframe ... %s" % dataframe.shape[1])
non_scaled_df = dataframe.copy()

print("... Scaling dataframe ===>")
dataframe = (dataframe-dataframe.min())/(dataframe.max()-dataframe.min())

In [ ]:
# non-scaled data
ploter(non_scaled_df,"SMOOTHED FILTERED DATA", 20, code)
non_scaled_df = non_scaled_df.drop([code], axis=1)

In [ ]:
# scaled data 
ploter(dataframe,"SMOOTHED FILTERED DATA (SCALED)", 20, code)

## 3. MODEL CONSTRUCTION

Next, we will develop a predictive model for the selected diagnosis code. The modeling process consists of two main phases:

- 1. Construction of the optimal model M at time t: In this phase, the goal is to build and train the best-performing model using all available information up to time t. This model will serve as a baseline for subsequent comparisons.
- 2. Testing of model M across different time lags, and construction of new models at times t - i: Here, we assess the robustness and generalizability of model M by applying it retrospectively to earlier time points (i.e., t - i) and comparing its performance with newly trained models specific to those earlier intervals.



In [ ]:
def metrics_calculation(models,y_train,y_test,Y_PREDICTIONS_train,Y_PREDICTIONS_test,params):

    F1 = []
    F2 = []
    p_values1 = []
    p_values2 = []
    n_vars = []
    MAPE_ts = []
    MAPE_tr = []
    RMSE_tr = []
    RMSE_ts = []
    pars = []


    for i in range(len(models) - 1):
        n = int(models[i].summary().tables[0].data[5][1])

        k1 = int(models[i].summary().tables[0].data[7][1])
        k2 = int(models[i+1].summary().tables[0].data[7][1])

        rss1 = models[i].ssr
        rss2 = models[i+1].ssr

        # f stat internet
        Fstat = ((rss1-rss2)/(k2-k1))/(rss2/(n-k2))
        p_value1 = 1-scipy.stats.f.cdf(Fstat, k2-k1, n-k2-1)

        #f stat paper
        sse2 = np.sum((Y_PREDICTIONS_train[i+1].values - y_train)**2)
        ssr1 = np.sum((Y_PREDICTIONS_train[i].values - y_train.mean())**2)
        ssr2 = np.sum((Y_PREDICTIONS_train[i+1].values - y_train.mean())**2)

        Fstat2 = ((ssr2-ssr1)/(k2-k1))/(sse2/(n-k2-1))
        p_value2 = 1-scipy.stats.f.cdf(Fstat2, k2-k1, n-k2-1)

        # mape
        mape_tr = mean_absolute_error(y_train, Y_PREDICTIONS_train[i])*100
        mape_ts = mean_absolute_error(y_test, Y_PREDICTIONS_test[i])*100

        #rmse
        rmse_tr = math.sqrt(mean_squared_error(y_train, Y_PREDICTIONS_train[i]))
        rmse_ts = math.sqrt(mean_squared_error(y_test, Y_PREDICTIONS_test[i]))

        F1.append(Fstat)
        F2.append(Fstat2)
        p_values1.append(p_value1)
        p_values2.append(p_value2)
        n_vars.append(i)
        MAPE_ts.append(mape_ts)
        MAPE_tr.append(mape_tr)
        RMSE_ts.append(rmse_ts)
        RMSE_tr.append(rmse_tr)
        pars.append(params[i])
    
    res = pd.DataFrame()
    res['number_of_variables'] = n_vars
    res["F1 (complex vs. intercept)"] = F1
    res['p-value (F1)'] = p_values1
    res["F2 (Complex vs. anterior)"] = F2
    res['p-value (F2)'] = p_values2
    res['MAPE train'] = MAPE_tr
    res['MAPE test'] = MAPE_ts
    res['RMSE train'] = RMSE_tr
    res['RMSE test'] = RMSE_ts
    res['predictors'] = pars
    
    return res

def models_training(model_vars,code, corr,max_iters):
    
    print(">>> Training models for predicting variable ... "+code)
    
    Y_PREDICTIONS_test = []
    Y_PREDICTIONS_train = []
    predictors = []
    models = []
    
    corr = corr[0:max_iters]

    # train test split ------------------------------------------------
    split_index = round(len(model_vars)*0.8)
    split_date = model_vars.index[split_index]

    df_train = model_vars.loc[model_vars.index <= split_date].copy()
    df_test = model_vars.loc[model_vars.index > split_date].copy()

    y_train = df_train[code].values
    y_test = df_test[code].values

    for i in range(1,len(corr)):

        if i % 25 == 0:
            print(">>> Iteration %s" % str(i))
        

        X_train = df_train
        X_train = X_train[corr.index[0:i].tolist()]

        X_test = df_test
        X_test = X_test[corr.index[0:i].tolist()]

        ols_model = sm.OLS(y_train,X_train)
        ols_results = ols_model.fit()
        models.append(ols_results)
        predictors.append(",".join(pd.DataFrame(ols_results.params).index.tolist()))

        y_pred_train = ols_results.predict(X_train)
        y_pred_test = ols_results.predict(X_test)
        Y_PREDICTIONS_test.append(y_pred_test)
        Y_PREDICTIONS_train.append(y_pred_train)
    

    print(">>> ALL MODELS -----------------------------------------")
    
    # plot all models
    fig = plt.figure()
    fig.suptitle(code+' prediction')
    lines = [] 
    for i in range(len(corr)-1):
        lines += plt.plot(df_test.index, Y_PREDICTIONS_test[i], label='%s iteration' % i)
    lines += plt.plot(df_test.index, y_test, 'go-', label='Actual Cov_19 diagnoses')
    plt.xlabel('Date')
    plt.ylabel('n diagnoses')
    plt.show()
    
    
    # metrics 
    df = metrics_calculation(models,y_train,y_test,Y_PREDICTIONS_train,Y_PREDICTIONS_test,predictors)

    sns.lineplot(data=df[['number_of_variables',"F1 (complex vs. intercept)","F2 (Complex vs. anterior)"]].melt(id_vars=['number_of_variables']),x='number_of_variables',y='value',hue="variable")
    plt.show()
    sns.lineplot(data=df[['number_of_variables',"p-value (F1)","p-value (F2)"]].melt(id_vars=['number_of_variables']),x='number_of_variables',y='value',hue="variable")
    plt.show()

    # models +++ interessants -------------------------
    interesting_models = [] # guardats
    n_int = 3               # fins a 3 models que passin el pval
    pval = 0.1              # pval
    c = 0
    for i in range(len(df)):
        if df['p-value (F1)'][i] >= pval:
            c += 1
            if c <= 3:
                interesting_models.append(i)
            else:
                pass
        else:
            pass
    
    df = df[0:max(interesting_models)+1]
    print(">>> BEST INTERESTING MODELS -----------------------------------------")
    display(df)
    
    # plot interesting
    fig = plt.figure()
    fig.suptitle(code+' prediction')
    lines = [] 
    for i in range(len(interesting_models)):
        lines += plt.plot(df_test.index, Y_PREDICTIONS_test[interesting_models[i]], label='%s iteration' % str(interesting_models[i]))

    lines += plt.plot(df_test.index, y_test, 'go-', label='Actual diagnoses')
    plt.xlabel('Date')
    plt.ylabel('n diagnoses')
    labels = [l.get_label() for l in lines]
    plt.legend(lines, labels) 
    plt.show()
    
    #### BEST ABSOLUTE MODEL

    print(">>> ABSOLUTE BEST MODEL -----------------------------------------")
    print(models[min(interesting_models)].summary())    
    
    df['BEST_MODEL'] = "NO"
    df.loc[min(interesting_models),"BEST_MODEL"] = "YES"
    
    # plot interesting
    fig = plt.figure()
    fig.suptitle(code+' prediction')
    lines = [] 
    lines += plt.plot(df_test.index, Y_PREDICTIONS_test[min(interesting_models)], label='%s iteration' % str(min(interesting_models)))
    lines += plt.plot(df_test.index, y_test, 'go-', label='Actual diagnoses')
    plt.xlabel('Date')
    plt.ylabel('n diagnoses')
    labels = [l.get_label() for l in lines]
    plt.legend(lines, labels) 
    plt.show()
    
    display(df[0:min(interesting_models)+1])
    
    return df

### A. Real-time prediction

In [ ]:
max_iters = 601

# sort absolute correations between variables and cov19
smot_corr = non_scaled_df.corrwith(smoothed[code]).sort_values(ascending=False, key=abs)

df = models_training(dataframe,code,smot_corr,max_iters)

### B. Lagged prediction

In [ ]:
max_iters = 75
max_lag = 30 

In [ ]:
smot_corr = non_scaled_df.corrwith(smoothed[code]).sort_values(ascending=False, key=abs)

# A. initial model (on-time prediction) ----------------
df_init = models_training(dataframe,code,smot_corr,max_iters)
BEST = df_init[df_init["BEST_MODEL"] == "YES"]
BEST["LAG"] = 0 

for i in range(1,max_lag+1):
    dat_lag = dataframe.copy()
    # add lag
    dat_lag[code] = dat_lag[code].shift(-i)
    dat_lag = dat_lag.dropna(subset=[code])
    
    # OBTENIR EL MILLOR MODEL
    df_init = models_training(dat_lag,code,smot_corr,max_iters)
    best = df_init[df_init["BEST_MODEL"] == "YES"]
    best["LAG"] = i
    BEST = pd.concat([BEST,best]).reset_index(drop=True)


In [ ]:
display(BEST)

In [ ]:
BEST.to_excel("BEST_features_NOSMOOTH_timeseries350.xlsx", index=False, engine='openpyxl')

### EVALUATION METRICS ANALYSIS

#### A. MAPE

In [ ]:
dff = BEST[["MAPE train", "MAPE test", "LAG"]]
sns.set(rc={'figure.figsize':(20,8)})
sns.lineplot(data=dff.replace('nan', float('nan')).melt(id_vars=['LAG']),x='LAG', y='value', hue='variable').set(title="MAPE ANALYSISIS IN LAGGED PREDICTION")
plt.axvline(7, 0,BEST[["MAPE train", "MAPE test"]].max(0).max(0), color="red")
plt.axvline(14, 0,BEST[["MAPE train", "MAPE test"]].max(0).max(0), color="red")
plt.axvline(21, 0,BEST[["MAPE train", "MAPE test"]].max(0).max(0), color="red")
plt.axvline(28, 0,BEST[["MAPE train", "MAPE test"]].max(0).max(0), color="red")

#### B. RMSE

In [ ]:
dff = BEST[["RMSE train", "RMSE test", "LAG"]]
sns.set(rc={'figure.figsize':(20,8)})
sns.lineplot(data=dff.replace('nan', float('nan')).melt(id_vars=['LAG']),x='LAG', y='value', hue='variable').set(title="RMSE ANALYSISIS IN LAGGED PREDICTION")
plt.axvline(7, 0,BEST[["RMSE train", "RMSE test", "LAG"]].max(0).max(0), color="red")
plt.axvline(14, 0,BEST[["RMSE train", "RMSE test", "LAG"]].max(0).max(0), color="red")
plt.axvline(21, 0,BEST[["RMSE train", "RMSE test", "LAG"]].max(0).max(0), color="red")
plt.axvline(28, 0,BEST[["RMSE train", "RMSE test", "LAG"]].max(0).max(0), color="red")

#### C. F vals

In [ ]:
dff = BEST[["F1 (complex vs. intercept)", "p-value (F1)", "F2 (Complex vs. anterior)", "p-value (F2)", "LAG"]]
sns.set(rc={'figure.figsize':(20,8)})
sns.lineplot(data=dff.replace('nan', float('nan')).melt(id_vars=['LAG']),x='LAG', y='value', hue='variable').set(title="F vals ANALYSISIS IN LAGGED PREDICTION")
plt.axvline(7, 0,BEST[["F1 (complex vs. intercept)", "p-value (F1)", "F2 (Complex vs. anterior)", "p-value (F2)",]].max(0).max(0), color="red")
plt.axvline(14, 0,BEST[["F1 (complex vs. intercept)", "p-value (F1)", "F2 (Complex vs. anterior)", "p-value (F2)",]].max(0).max(0), color="red")
plt.axvline(21, 0,BEST[["F1 (complex vs. intercept)", "p-value (F1)", "F2 (Complex vs. anterior)", "p-value (F2)",]].max(0).max(0), color="red")
plt.axvline(28, 0,BEST[["F1 (complex vs. intercept)", "p-value (F1)", "F2 (Complex vs. anterior)", "p-value (F2)",]].max(0).max(0), color="red")

## C. Prediction lagged first model

In [ ]:
Y_PREDICTIONS_test = []  # training results
Y_PREDICTIONS_train = [] # testing results
no_lag_predictors = True

for lagging in range(0,max_lag):
    
    # predictors del millor model per a lag == lagging (0,30) o inicial
        # varien en funció del model
    if no_lag_predictors == True:
        predictors = BEST.loc[0,"predictors"].split(",")
    else:
        predictors = BEST.loc[lagging,"predictors"].split(",")
    
    # copia de la bbbdd
    df = dataframe.copy()
    split_index = round(len(df)*0.8) 
    df = df[[code]+predictors]
    
    # Realitzem una copia lagged de la bbdd
    if lagging > 0:
        df[code] = df[code].shift(-lagging)
        df = df.dropna(subset=[code])
    print("TRAINING ROWS post lagging % s" % str(df.shape))

    
    #train_test_split
    split_date = df.index[split_index-lagging]
    df_train = df.loc[df.index <= split_date].copy()
    df_test = df.loc[df.index > split_date].copy()
    y_train = df_train[code].values
    y_test = df_test[code].values
    X_train = df_train
    X_train = X_train[predictors]
    X_test = df_test
    X_test = X_test[predictors]

    # model training
    ols_model = sm.OLS(y_train,X_train)
    ols_results = ols_model.fit()

    #populate vars
    y_pred_train = ols_results.predict(X_train)
    y_pred_test = ols_results.predict(X_test)
    Y_PREDICTIONS_test.append(y_pred_test)
    Y_PREDICTIONS_train.append(y_pred_train)

# plot all models
fig = plt.figure()
fig.suptitle(code+' prediction')
lines = [] 

for i in range(len(Y_PREDICTIONS_test)):#-1):
    lines += plt.plot(df_test.index, Y_PREDICTIONS_test[i], label='%s day lagging' % str(i+1))
lines += plt.plot(df_test.index, y_test, 'go-', label='Actual %s diagnoses' % code)
labels = [l.get_label() for l in lines]
plt.legend(lines, labels) 
plt.xlabel('Date')
plt.ylabel('n diagnoses')
plt.show()

In [ ]:
BEST